# **Otimização - Toy Experiment: Mitigação de Crosstalk**
## **Notebook de Otimização**

**Disciplina:** PPGEE0016 - Otimização

**Alunos:** André Paiva, Josias Souza, Victor Emanuel Paz

In [ ]:
import sys
sys.path.insert(1, "../crosstalk/")

from time import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import rc
import seaborn as sns

from functions import *
from XTconstants import *

sns.set_context("paper", font_scale=2.0)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
# Setting a random seed for reproducibility. Using a fixed seed value (5) ensures that the random numbers generated
# are the same across different runs of the code. This is important for verifying results, as it allows
# for consistent experimentation and comparison of outcomes.
np.random.seed(5)

In [ ]:
df = pd.read_csv("./dataset/data.csv", index_col=0)
df

In [ ]:
def func_s(params, n_samples=4, sampling_period=25):
    """
    Function to approximate samples from signal contaminated by crosstalk.

    Parameters
    ----------
    ``params``: Array of dictionaries containing E, tau, E_x, tau_x.
    ``n_samples``: Number of samples (default is 4).
    ``sampling_period``: Time distance between samples (default is 25ns).

    Returns
    -------
    Value from sample of approximated function according to given parameters.

    """

    clean_signal = np.array([params[i]["E"]*np.array(
                        [cellFunction(sampling_period*(j+1)*46.74/params[i]["tau"] + params[i]["sample_delay"]) for j in range(n_samples)]
                    ) for i in range(len(params))])
    xt_signal = np.array([params[i]["E_x"]*np.array(
                        [XTalk(sampling_period*(j+1)*32.16/params[i]["tau_x"] + params[i]["sample_delay"]) for j in range(n_samples)]
                    ) for i in range(len(params))])
    return clean_signal + xt_signal

In [ ]:
def func_mse(signals_df, params, populacao, n_samples=4):
    func_signals = func_s(params, n_samples=n_samples, sampling_period=signals_df["sampling_period"][0])
    samples = np.array([signals_df[f"S_{i}"] for i in range(1, n_samples+1)]).T
    func_signals_matrix = np.array([func_signals for i in range(samples.shape[0])])
    samples_matrix = np.transpose(np.array([samples for i in range(populacao)]), (1, 0, 2))
    square_error = (samples_matrix - func_signals_matrix)**2
    mse_matrix = np.sum(square_error, axis=2)/square_error.shape[2]
    return mse_matrix

In [ ]:
def genetico(signals_df, funcao=func_mse, populacao=50, iteracoes=30, p_recomb=0.3, p_mutacao = 0.15):
    E_init = np.abs(np.random.normal(loc=90000, scale=80000, size=populacao))
    tau_init = np.abs(np.random.normal(loc=60, scale=10, size=populacao))
    E_x_init = np.abs(np.random.normal(loc=9000, scale=8000, size=populacao))
    tau_x_init = np.abs(np.random.normal(loc=60, scale=10, size=populacao))
    samp_delay_init = np.abs(np.random.normal(loc=3, scale=4, size=populacao))
    x_matrix = np.stack([E_init, tau_init, E_x_init, tau_x_init, samp_delay_init], axis=1)
    avg_fitness = np.array([])
    min_fitness = np.array([])
    col = ['E', 'tau', 'E_x', 'tau_x', 'sample_delay', 'fitness', 'iter']
    df_evolution = pd.DataFrame(columns=col)
    x_pop = np.copy(x_matrix)
    time_init = time()
    best = [{"E" : -np.inf, "tau": -np.inf, "E_x": -np.inf, "tau_x": -np.inf, "sample_delay": -np.inf} for i in range(signals_df.shape[0])]
    for i in range(iteracoes):
        # Cálculo de Fitness e inserção no dataframe
        x_params = [{"E": candidato[0],
                     "tau": candidato[1],
                     "E_x": candidato[2],
                     "tau_x": candidato[3],
                     "sample_delay": candidato[4]} for candidato in x_pop]
        mse_matrix = -funcao(signals_df, x_params, populacao)
        x_fitness = np.sum(mse_matrix, axis=0).reshape(-1, 1)
        iter_atual = np.tile([i], x_matrix.shape[0]).reshape(-1, 1)
        df_data = np.concatenate((x_pop, x_fitness, iter_atual), axis=1)
        df_evolution = pd.concat([df_evolution, pd.DataFrame(df_data, columns=col)]).reset_index(drop=True)
        # Captura de fitness médio e fitness mínimo por iteração
        avg_fitness = np.append(avg_fitness, np.mean(x_fitness))
        min_fitness = np.append(min_fitness, np.min(x_fitness))
        # Sorting Crescente
        best = [{"E" : x_pop[np.argmax(mse_matrix[i]), 0],
                "tau": x_pop[np.argmax(mse_matrix[i]), 1],
                "E_x": x_pop[np.argmax(mse_matrix[i]), 2],
                "tau_x": x_pop[np.argmax(mse_matrix[i]), 3],
                "sample_delay": x_pop[np.argmax(mse_matrix[i]), 4]} for i in range(signals_df.shape[0])]
        x_sort = np.argsort(x_fitness, axis=0).reshape(-1)
        x_fitness = x_fitness[x_sort]
        x_pop = x_pop[x_sort]
        # Cálculo de probabilidades de seleção
        prob_num = np.array([np.sum(np.arange(1, i+1)) for i in range(1, x_matrix.shape[0]+1)])
        prob_den = np.sum(np.arange(1, x_fitness.shape[0]+1))
        prob = prob_num/prob_den
        # Seleção de elementos
        selecao_prob = np.random.rand(np.ceil(x_fitness.shape[0]/2).astype(np.int32), 2)
        index_selecao_prob = np.searchsorted(prob, selecao_prob, side='right')
        # Recombinação
        new_pop = np.empty((0, x_pop.shape[1]))
        for j in index_selecao_prob:
            candidato1 = x_pop[j[0]]
            candidato2 = x_pop[j[1]]
            recomb_mask = np.random.rand(candidato1.shape[0])
            recomb_mask = np.array([p <= p_recomb for p in recomb_mask])
            novo_candidato1 = np.copy(candidato1)
            np.putmask(novo_candidato1, recomb_mask, candidato2)
            novo_candidato1 = novo_candidato1.reshape((1, -1))
            novo_candidato2 = np.copy(candidato2)
            np.putmask(novo_candidato2, recomb_mask, candidato1)
            novo_candidato2 = novo_candidato2.reshape((1, -1))
            new_pop = np.concatenate((new_pop, novo_candidato1, novo_candidato2), axis=0)
        new_pop = new_pop[:x_pop.shape[0]]
        # Mutação
        E_init = np.abs(np.random.normal(loc=90000, scale=80000, size=populacao))
        tau_init = np.abs(np.random.normal(loc=60, scale=10, size=populacao))
        E_x_init = np.abs(np.random.normal(loc=9000, scale=8000, size=populacao))
        tau_x_init = np.abs(np.random.normal(loc=60, scale=10, size=populacao))
        samp_delay_init = np.abs(np.random.normal(loc=3, scale=4, size=populacao))
        mut_gen = np.stack([E_init, tau_init, E_x_init, tau_x_init, samp_delay_init], axis=1)
        mut_mask = np.random.rand(new_pop.shape[0], new_pop.shape[1])
        mut_mask = np.array([p <= p_mutacao for p in mut_mask])
        np.putmask(new_pop, mut_mask, mut_gen)
        # Nova População
        x_pop = np.copy(new_pop)
        if(i == iteracoes - 1):
             # Cálculo de Fitness e inserção no dataframe
            x_params = [{"E": candidato[0],
                         "tau": candidato[1],
                         "E_x": candidato[2],
                         "tau_x": candidato[3],
                         "sample_delay": candidato[4]} for candidato in x_pop]
            mse_matrix = -funcao(signals_df, x_params, populacao)
            x_fitness = np.sum(mse_matrix, axis=0).reshape(-1, 1)
            iter_atual = np.tile([i], x_matrix.shape[0]).reshape(-1, 1)
            df_data = np.concatenate((x_pop, x_fitness, iter_atual), axis=1)
            df_evolution = pd.concat([df_evolution, pd.DataFrame(df_data, columns=col)]).reset_index(drop=True)
            # Captura de fitness médio e fitness mínimo por iteração
            avg_fitness = np.append(avg_fitness, np.mean(x_fitness))
            min_fitness = np.append(min_fitness, np.min(x_fitness))

    elapsed_time = (time() - time_init) * 1000
    fitness_metrics = np.concatenate(([avg_fitness], [min_fitness]), axis = 0).transpose()
    df_fitness = pd.DataFrame(fitness_metrics)
    #df_fitness['Aptidao'] = ['Media', 'Minima']
        
    return best, df_evolution, df_fitness, elapsed_time

In [ ]:
best, df_evolution, df_fitness, elapsed_time = genetico(df)

In [ ]:
df_evolution["Gerações"] = df_evolution["iter"]
df_evolution["Aptidão"] = df_evolution["fitness"]
sns.lineplot(df_evolution, x='Gerações', y='Aptidão')
plt.tight_layout()
plt.savefig("fitness_paralelo.pdf")

In [ ]:
signal_index=10
df_est_comparison = pd.DataFrame(data=[["E", best[signal_index]["E"], df["E"][signal_index]],
                                 ["Tau", best[signal_index]["tau"], df["tau"][signal_index]],
                                 ["E_x", best[signal_index]["E_x"], df["E_x"][signal_index]],
                                 ["Tau_x", best[signal_index]["tau_x"], df["tau_x"][signal_index]],
                                 ["Sampling Delay", best[signal_index]["sample_delay"], df["delay_sampling"][signal_index]]],
                                 columns=["Parâmetro", "Estimado", "Esperado"])

df_est_comparison

In [ ]:
plt.plot([df["delay_sampling"][signal_index] + (25*i) for i in range(1, 5)],
         [df[f"S_{i}"][signal_index] for i in range(1, 5)], marker='o', label="Samples")

ETrue_est = np.array([best[signal_index]["E"]*cellFunction(t*46.74/best[signal_index]["tau"]) for t in range(600)])
XT_est = np.array([best[signal_index]["E_x"]*XTalk(t*32.16/best[signal_index]["tau_x"]) for t in range(600)])
signal_est = ETrue_est + XT_est

ETrue_gt = np.array([df["E"][signal_index]*cellFunction(t*46.74/df["tau"][signal_index]) for t in range(600)])
XT_gt = np.array([df["E_x"][signal_index]*XTalk(t*32.16/df["tau_x"][signal_index]) for t in range(600)])
signal_gt = ETrue_gt + XT_gt

#Gráfico Comparativo
plt.plot(range(600), signal_est, label="Signal with estimated parameters")
plt.plot(range(600), signal_gt, label="Ground Truth Signal")
plt.legend()
plt.show()


In [ ]:
signal_index=10

E_diff = pd.DataFrame(np.array([(best[i]["E"] - df["E"][i]) for i in range(df.shape[0])]))
tau_diff = pd.DataFrame(np.array([(best[i]["tau"] - df["tau"][i]) for i in range(df.shape[0])]))
E_x_diff = pd.DataFrame(np.array([(best[i]["E_x"] - df["E_x"][i]) for i in range(df.shape[0])]))
tau_x_diff = pd.DataFrame(np.array([(best[i]["tau_x"] - df["tau_x"][i]) for i in range(df.shape[0])]))
sample_delay_diff = pd.DataFrame(np.array([(best[i]["sample_delay"] - df["delay_sampling"][i]) for i in range(df.shape[0])]))

E_diff["Param"] = "E"
tau_diff["Param"] = "tau"
E_x_diff["Param"] = "E_x"
tau_x_diff["Param"] = "tau_x"
sample_delay_diff["Param"] = "sampling_delay"

df_comparison = pd.concat([E_diff, E_x_diff, tau_diff, tau_x_diff, sample_delay_diff])
df_comparison["Erro"] = df_comparison[0]
df_comparison["Parâmetro"] = df_comparison["Param"]
df_comparison.groupby("Parâmetro")["Erro"].agg(["mean", "std"])

In [ ]:
df_comparison.groupby("Parâmetro")["Erro"].agg(["mean", "std"]).to_latex()

In [ ]:
df_diff = pd.concat([E_diff, E_x_diff])
df_diff

fig, ax = plt.subplots(2, 1, figsize=(10, 6))
sns.boxplot(data=df_diff, x=0, y="Param", ax=ax[0], hue="Param")

#ax[0].set_title("Erro de estimação de energia - Otimização Paralela")
ax[0].set_xlabel("Energia (MeV)")
ax[0].set_ylabel(None)
ax[0].set_yticklabels(["E", f"${{E_x}}$"])

df_diff = pd.concat([tau_diff, tau_x_diff, sample_delay_diff])
df_diff

sns.boxplot(data=df_diff, x=0, y="Param", ax=ax[1], hue="Param")
#ax[1].set_title("Erro de estimação de tempo - Otimização Paralela")
ax[1].set_xlabel("Tempo (ns)")
ax[1].set_ylabel(None)
ax[1].set_yticklabels([r'$\tau$', r'$\tau_x$', "sample_delay"])
plt.tight_layout()
fig.savefig("boxplot_paralelo.pdf")

In [ ]:
for k in range(df.shape[0]):
    plt.plot([df["delay_sampling"][k] + (25*i) for i in range(1, 5)],
            [df[f"S_{i}"][k] for i in range(1, 5)], marker='o', label="Samples")

    ETrue_est = np.array([best[k]["E"]*cellFunction(t*46.74/best[k]["tau"]) for t in range(600)])
    XT_est = np.array([best[k]["E_x"]*XTalk(t*32.16/best[k]["tau_x"]) for t in range(600)])
    signal_est = ETrue_est + XT_est

    ETrue_gt = np.array([df["E"][k]*cellFunction(t*46.74/df["tau"][k]) for t in range(600)])
    XT_gt = np.array([df["E_x"][k]*XTalk(t*32.16/df["tau_x"][k]) for t in range(600)])
    signal_gt = ETrue_gt + XT_gt

    #Gráfico Comparativo
    plt.plot(range(600), signal_est, label="Signal with estimated parameters")
    plt.plot(range(600), signal_gt, label="Ground Truth Signal")
    plt.legend()
    plt.show()
